This notebook aims at merging all the shattered tweet datafiles (available at [https://crisisnlp.qcri.org/humaid_dataset.html#](https://crisisnlp.qcri.org/humaid_dataset.html#)) into one CSV file. This will come in handy for further data exploration tasks.

In [1]:
# Import packages
import re
import os
import numpy as np
import pandas as pd
import tweepy
import json
import time

In [2]:
# Select the folder list that contain information
folder_list1 = [handle for handle in os.listdir('assets/events_set1') if re.match('.*_(\d){4}',handle)]
folder_list2 = [handle for handle in os.listdir('assets/events_set2') if re.match('.*_(\d){4}',handle)]

In [3]:
# Aggregate all the datasets
df_train = pd.DataFrame()
df_dev = pd.DataFrame()
df_test = pd.DataFrame()

for i,folder in enumerate([folder_list1,folder_list2]):
    for filename in folder:
        search_string=re.search('(?P<place>.*)_(?P<disaster>.*)_(?P<year>.*)',filename)
        df_train_sub = pd.read_table('assets/events_set'+str(i+1)+'/'+filename+'/'+filename+'_train.tsv',delimiter = '\t',header=0)
        df_dev_sub = pd.read_table('assets/events_set'+str(i+1)+'/'+filename+'/'+filename+'_dev.tsv',delimiter = '\t',header=0)
        df_test_sub = pd.read_table('assets/events_set'+str(i+1)+'/'+filename+'/'+filename+'_dev.tsv',delimiter = '\t',header=0)
        df_train_sub['place'] = search_string['place']
        df_dev_sub['place'] = search_string['place']
        df_test_sub['place'] = search_string['place']
        df_train_sub['disaster'] = search_string['disaster']
        df_dev_sub['disaster'] = search_string['disaster']
        df_test_sub['disaster'] = search_string['disaster']
        df_train_sub['year'] = int(search_string['year'])
        df_dev_sub['year'] = int(search_string['year'])
        df_test_sub['year'] = int(search_string['year'])
        df_train = pd.concat([df_train,df_train_sub])
        df_dev = pd.concat([df_dev,df_dev_sub])
        df_test = pd.concat([df_test,df_test_sub])
df_train['tweet_id'] = df_train['tweet_id'].astype(str)
df_dev['tweet_id'] = df_dev['tweet_id'].astype(str)
df_test['tweet_id'] = df_test['tweet_id'].astype(str)

In [4]:
# We then try to retrieve the tweet info using tweepy.
# consumer_key = '6oDNcDJCUNB8I9jl0EaFFPAH6'
# consumer_secret = 'hbmi7b3kn4N2rOdHbooLmGs25bFAArYKJpfZUPybOdnreosoJy'
# access_token = '1377468317351485442-YVzWG5leSjfHh8lYpObb7bCDTDQ60w'
# access_token_secret = 'lI4gF5mFsIYgyIBX9SpZ5fm1Q5WdiQKsHKgX0Tm6olzJg'

consumer_key = 'PxIp15u5Fpp2LnbZDgis2zQJx'
consumer_secret = 'HXuxvCM23jftOh6Y078tncGwU1Bb3CtbUKtxMEYsIZwx7zhBvi'
access_token = '1364901780388147204-SLV0VPgti7TXWnuWYzjunFlfGLnSmv'
access_token_secret = 'D0nS6lfYfKDZH4wpWxWqw9ylyZ1W2Rl8cjXaMbXMk2Upz'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth) 

In [8]:
# Scrape info on the test set
df_test_scrape = pd.DataFrame()
for tweet_id in df_test.iloc[0:3]['tweet_id']:
    try:
        df_test_scrape = df_test_scrape.append(api.get_status(id=tweet_id)._json,ignore_index=True)
    except tweepy.TweepError:
        time.sleep(16*60)
        df_test_scrape = df_test_scrape.append(api.get_status(id=tweet_id)._json,ignore_index=True)

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,source,text,truncated,user,retweeted_status
0,None,None,Thu May 19 18:16:11 +0000 2016,"{'hashtags': [{'text': 'AlbertaStrong', 'indic...",0.0,0.0,None,7.333606e+17,733360620339257344,None,...,{'created_at': 'Thu May 19 18:00:52 +0000 2016...,7.333568e+17,733356762875592704,0.0,0.0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",.@GreenABEnergy How can @AirworksCanada assist...,0.0,"{'id': 3905236364, 'id_str': '3905236364', 'na...",NaN
1,None,None,Mon May 09 03:58:37 +0000 2016,"{'hashtags': [{'text': 'PrayForCanada', 'indic...",0.0,0.0,None,7.295209e+17,729520926795157504,None,...,NaN,NaN,NaN,1.0,0.0,"<a href=""http://twitter.com/download/android"" ...",RT @katvondawn: Thoughts &amp; prayers going t...,0.0,"{'id': 60437904, 'id_str': '60437904', 'name':...",{'created_at': 'Sat May 07 20:32:08 +0000 2016...
2,None,None,Thu May 12 12:41:05 +0000 2016,"{'hashtags': [], 'symbols': [], 'user_mentions...",1.0,0.0,None,7.307396e+17,730739571739271168,None,...,NaN,NaN,NaN,0.0,0.0,"<a href=""http://www.hootsuite.com"" rel=""nofoll...",Glacier Farm Media pledges $50K in support for...,0.0,"{'id': 2320930195, 'id_str': '2320930195', 'na...",NaN


In [9]:
df_train.to_csv('assets/df_train.csv')
df_dev.to_csv('assets/df_dev.csv')
df_test.to_csv('assets/df_test.csv')